In [3]:
l = -760
h = 585
wh = 100
wl = 760
profit_h = (wh / 100) * h - wl
profit_l = (-wl / l) * 100 - wh
print(f'Profit H is {profit_h} while profit L is {profit_l}. the ratio of wagers \n wh / wl is {wh / wl}')

Profit H is -175.0 while profit L is 0.0. the ratio of wagers 
 wh / wl is 0.13157894736842105


In [132]:
import requests
from datetime import datetime
import pandas as pd

# Get Game ID List:
api_endpoint = 'https://api-web.nhle.com/v1/schedule/'

start_date = '2020-08-01'
end_date = '2023-09-26'

date_range = pd.date_range(start_date, end_date, 
              freq='W').strftime("%G-%m-%d").tolist()

game_id_list = []
for date in date_range:
    api_link = f"https://api-web.nhle.com/v1/schedule/{date}"
    response = requests.get(api_link)
    data = response.json()
    for day in data['gameWeek']:
        for game in day['games']:
            game_id_list.append(game['id'])
            
# Ensure no duplicates
game_id_list = set(game_id_list)

# Define the NHL API endpoint for the schedule endpoint
game_stats = []
game_team_stats = []
game_skater_stats = []
game_goalie_stats = []

for GAMEID in game_id_list:
    api_url = f"https://api-web.nhle.com/v1/gamecenter/{GAMEID}/boxscore"
    response = requests.get(api_url)
    data = response.json()
    
    outcome_prefix = None if data["gameState"] == 'FUT' else 'home win ' if data["homeTeam"]["score"] > data["awayTeam"]["score"] else 'away win'
    outcome_suffix = '' if data["gameState"] == 'FUT' else'REG' if len(data['boxscore']['linescore']['byPeriod']) == 3 else 'OT'
    
    game_stats_dict = {
        "game_id" : GAMEID,
        "season" : data["season"],
        "type" : data["gameType"],
        "date_time_GMT" : data["startTimeUTC"],
        "away_team_id" : str(data["awayTeam"]["id"]),
        "home_team_id" : str(data["homeTeam"]["id"]),
        "away_goals" : data["awayTeam"]["score"] if data["gameState"] != 'FUT' else None,
        "home_goals" : data["homeTeam"]["score"] if data["gameState"] != 'FUT' else None,
        "outcome" : outcome_prefix + outcome_suffix,
        "venue": data["venue"],
        "game_state" : data["gameState"]
    }
    
    game_stats.append(game_stats_dict)
    
    # Only append stats for games that have started
    if datetime.strptime(data["startTimeUTC"], "%Y-%m-%dT%H:%M:%SZ") < datetime.now():
        home_team_stats_dict = {
            "game_id" : GAMEID,
            "team_id" : str(data["homeTeam"]["id"]),
            "HoA" : "home",
            "won" : None if data["homeTeam"].get("score") is None else True if data["homeTeam"].get("score") > data["awayTeam"].get("score") else False,
            "settled_in" : outcome_suffix,
            "head_coach" : data["boxscore"]["gameInfo"]["homeTeam"]["headCoach"],
            "goals" : data["homeTeam"]["score"],
            "shots" : data["homeTeam"].get('sog'),
            "hits" : data["homeTeam"]["hits"],
            "pim" : data["homeTeam"]["pim"],
            "blocks" : data["homeTeam"]["blocks"],
            "faceoffWinningPctg" : data["homeTeam"]["faceoffWinningPctg"],
            "powerPlayConversion" : data["homeTeam"]["powerPlayConversion"],
        }

        away_team_stats_dict = {
            "game_id" : GAMEID,
            "team_id" : str(data["awayTeam"]["id"]),
            "HoA" : "away",
            "won" : None if data["homeTeam"].get("score") is None else False if data["homeTeam"].get("score") > data["awayTeam"].get("score") else True,
            "settled_in" : outcome_suffix,
            "head_coach" : data["boxscore"]["gameInfo"]["awayTeam"]["headCoach"],
            "goals" : data["awayTeam"]["score"],
            "shots" : data["awayTeam"].get('sog'),
            "hits" : data["awayTeam"]["hits"],
            "pim" : data["awayTeam"]["pim"],
            "blocks" : data["awayTeam"]["blocks"],
            "faceoffWinningPctg" : data["awayTeam"]["faceoffWinningPctg"],
            "powerPlayConversion" : data["awayTeam"]["powerPlayConversion"],
        }

        for player_type in ['forwards', 'defense']:
            for player in data['boxscore']['playerByGameStats']['homeTeam'][player_type]:
                home_team_skater_stats_dict = {
                    "game_id" : GAMEID,
                    "player_id" : player['playerId'],
                    "team_id" : str(data["homeTeam"]["id"]),
                    "time_on_ice" : player['toi'],
                    "assists" : player['assists'],
                    "goals" : player['goals'],
                    "shots" : player['shots'],
                    "hits" : player['hits'],
                    "power_play_goals" : player['powerPlayGoals'],
                    "power_play_assists" : player['powerPlayPoints'] - player['powerPlayGoals'],
                }
                game_skater_stats.append(home_team_skater_stats_dict)

        for player_type in ['forwards', 'defense']:
            for player in data['boxscore']['playerByGameStats']['awayTeam'][player_type]:
                away_team_skater_stats_dict = {
                    "game_id" : GAMEID,
                    "player_id" : player['playerId'],
                    "team_id" : str(data["awayTeam"]["id"]),
                    "time_on_ice" : player['toi'],
                    "assists" : player['assists'],
                    "goals" : player['goals'],
                    "shots" : player['shots'],
                    "hits" : player['hits'],
                    "power_play_goals" : player['powerPlayGoals'],
                    "power_play_assists" : player['powerPlayPoints'] - player['powerPlayGoals'],
                }
                game_skater_stats.append(away_team_skater_stats_dict)

        for goalie in data['boxscore']['playerByGameStats']['homeTeam']['goalies']:
            home_team_goalie_stats_dict = {
                "game_id" : GAMEID,
                "player_id" : goalie['playerId'],
                "team_id" : str(data["homeTeam"]["id"]),
                "time_on_ice" : goalie['toi'],
                "pim" : goalie['pim'],
                "even_strength_shots_against" : goalie['evenStrengthShotsAgainst'],
                "saves_shots_against" : goalie['saveShotsAgainst'],
                "power_play_shots_against" : goalie['powerPlayShotsAgainst'],
                "goals_against" : goalie['goalsAgainst'],
            }
            if home_team_goalie_stats_dict['time_on_ice'] != '00:00':
                game_goalie_stats.append(home_team_goalie_stats_dict)

        for goalie in data['boxscore']['playerByGameStats']['awayTeam']['goalies']:
            away_team_goalie_stats_dict = {
                "game_id" : GAMEID,
                "player_id" : goalie['playerId'],
                "team_id" : str(data["awayTeam"]["id"]),
                "time_on_ice" : goalie['toi'],
                "pim" : goalie['pim'],
                "even_strength_shots_against" : goalie['evenStrengthShotsAgainst'],
                "saves_shots_against" : goalie['saveShotsAgainst'],
                "power_play_shots_against" : goalie['powerPlayShotsAgainst'],
                "goals_against" : goalie['goalsAgainst'],
            }
            if away_team_goalie_stats_dict['time_on_ice'] != '00:00':
                game_goalie_stats.append(away_team_goalie_stats_dict)


        # Append home and away stats
        game_team_stats.append(home_team_stats_dict)
        game_team_stats.append(away_team_stats_dict)

# Create a DataFrame from the extracted data
df_game = pd.DataFrame(game_stats)
df_game_team_stats = pd.DataFrame(game_team_stats)
df_game_skater_stats = pd.DataFrame(game_skater_stats)
df_game_goalie_stats = pd.DataFrame(game_goalie_stats)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [13]:
import os 
import pandas as pd
os.chdir('/Users/jdmcatee/Desktop/sports_betting/data/maintained_tables')

df = pd.read_csv('fct_maintained_game.csv')
df[df.game_state != 'FUT']['date_time_GMT'].max()

'2023-10-22T02:30:00Z'

In [8]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path_to_your_.json_credential_file"

client = bigquery.Client()
project = "proud-storm-399815"
dataset_id = "proud-storm-399815.fct_maintained_game"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("proud-storm-399815.fct_maintained_game.fct_maintained_game")
table = client.get_table(table_ref)

df = client.list_rows(table).to_dataframe()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
df